In [10]:
import requests
import pandas as pd
from pandas import json_normalize
import numpy as np
import json
from ast import literal_eval
from konlpy.tag import Twitter
import nltk
from nltk import tokenize
from collections import Counter

twit = Twitter()
payload = {'key1':'value1','key2':'value2'}
headers = {'Content-Type': 'application/json; charset=utf-8'}

r = requests.get('http://192.168.1.55:8000/api/train/user-novel-grade',params=payload, headers=headers)
rjson = r.json()



df = json_normalize(rjson['data'])

# df['has_review'] = df['has_review'].apply(lambda x: ','.join([str(i) for i in x]))
# df['avg_score'] = df['avg_score'].apply(lambda x: ','.join([str(i) for i in x]))

/home/so/anaconda3/envs/so/lib/python3.8/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [11]:
rjson

{'result': 'success',
 'data': [{'novel_id': 1, 'user_id': 2, 'grade': 5},
  {'novel_id': 1, 'user_id': 3, 'grade': 4.5},
  {'novel_id': 1, 'user_id': 4, 'grade': 4},
  {'novel_id': 1, 'user_id': 5, 'grade': 4},
  {'novel_id': 1, 'user_id': 6, 'grade': 5},
  {'novel_id': 1, 'user_id': 7, 'grade': 9},
  {'novel_id': 1, 'user_id': 8, 'grade': 2},
  {'novel_id': 1, 'user_id': 9, 'grade': 7.75},
  {'novel_id': 1, 'user_id': 10, 'grade': 3},
  {'novel_id': 1, 'user_id': 11, 'grade': 3.3333333333333335},
  {'novel_id': 1, 'user_id': 12, 'grade': 2},
  {'novel_id': 1, 'user_id': 13, 'grade': 5.75},
  {'novel_id': 1, 'user_id': 14, 'grade': 3.5},
  {'novel_id': 1, 'user_id': 16, 'grade': 7.5},
  {'novel_id': 1, 'user_id': 17, 'grade': 4.333333333333333},
  {'novel_id': 1, 'user_id': 18, 'grade': 6.5},
  {'novel_id': 1, 'user_id': 19, 'grade': 2},
  {'novel_id': 1, 'user_id': 20, 'grade': 9},
  {'novel_id': 1, 'user_id': 21, 'grade': 3},
  {'novel_id': 1, 'user_id': 22, 'grade': 2.5},
  {'novel

In [2]:
df = df.dropna()
df = df.loc[df.grade !=0]

sample_num = 30
unique_user_lst = list(np.unique(df['user_id']))
print("유저 수 : {}".format(len(unique_user_lst)))
print("sample_data : {}".format(sample_num))

유저 수 : 30
sample_data : 30


In [3]:
sample_user_idx = np.random.choice(len(unique_user_lst), sample_num, replace = False)
sample_user_lst = [unique_user_lst[idx] for idx in sample_user_idx]
df = df[df['user_id'].isin(sample_user_lst)] # 
df = df.reset_index(drop = True)

"""==========================================================================================="""

print(" 유저 아이디 : {}".format(unique_user_lst[:5]))
print(df)

 유저 아이디 : [2, 3, 4, 5, 6]
      novel_id  user_id  grade
0            1        2    5.0
1            1        3    4.5
2            1        4    4.0
3            1        5    4.0
4            1        6    5.0
...        ...      ...    ...
1002        35       27    9.5
1003        35       28    9.0
1004        35       29    2.5
1005        35       30    3.0
1006        35       31    4.0

[1007 rows x 3 columns]


In [4]:
# 2명 이상의 score 데이터가 있는 유저만 사용
df_count = df.groupby(['user_id']).count()
df['count'] = df.groupby('user_id')['user_id'].transform('count')
df = df[df['count'] > 1]

"""==========================================================================================="""
df

,novel_id,user_id,grade,count
0,1,2,5.0,33
1,1,3,4.5,34
2,1,4,4.0,34
3,1,5,4.0,33
4,1,6,5.0,35
...,...,...,...,...
1002,35,27,9.5,32
1003,35,28,9.0,33
1004,35,29,2.5,35
1005,35,30,3.0,33


In [5]:
from tqdm import tqdm

"""
train, test 데이터 생성 ( 딥러닝 학습,테스트용 )
1. 학습용 데이터 컬럼 추출 
2. 학습용 데이터, 테스트 데이터 생성
3. df_neg >> 유저 아이디 마다 하나의 Positive item(구매제품), N개의 Negative item(구매하지않은 제품)으로 구성된 테이블 생성
"""

df = df[['user_id', 'novel_id', 'grade']]

def mask_first(x) :
    result = np.ones_like(x)
    result[0] = 0
    return result

df_test = df.copy(deep = True)
df_train = df.copy(deep = True)

df_test = df_test.groupby(['user_id']).first()
df_test['user_id'] = df_test.index
df_test = df_test[['user_id','novel_id','grade']]
df_test = df_test.reset_index(drop=True)

mask = df.groupby(['user_id'])['user_id'].transform(mask_first).astype(bool)
df_train = df.loc[mask]

users = list(np.sort(df.user_id.unique()))
novels = list(np.sort(df.novel_id.unique()))

train_rows = df_train['user_id'].astype(int)
train_cols = df_train['novel_id'].astype(int)
train_values = list(df_train.grade)

train_uids = np.array(train_rows.tolist())
train_iids = np.array(train_cols.tolist())

real_rows = df['user_id'].astype(int)
real_cols = df['novel_id'].astype(int)
real_values = list(df.grade)

real_uids = np.array(real_rows.tolist())
real_iids = np.array(real_cols.tolist())

def get_all_negatives():

    df_neg = self.get_negatives(real_uids, real_iids, novels, df)
    df_neg.insert(0, 'users', pd.DataFrame(list(df_neg[0]))[0])
    df_neg.insert(1, 'novels', pd.DataFrame(list(df_neg[0]))[1])  
    del df_neg[0]
    ids = df_neg[df_neg['users']==user_id]
    list_ = ids[ids.columns[2:]].values
    list_ = np.array(ids[ids.columns[2:]].values).flatten().tolist()
    set(list_)
    return list_

def get_negatives(uids, iids, items, DataFrame) :
    
    """
    negative item 리스트 생성 함수
    negative 리스트 >> 유저별 구매하지 않은, 조회하지 않은, 평점을 달지 않은 데이터 시트 (테스트 및 상용화)
    """
    negativeList = []
    df_u = DataFrame['user_id'].values.tolist()
    df_i = DataFrame['novel_id'].values.tolist()
    
    df_ratings = list(zip(df_u,df_i))
    zipped = set(zip(uids,iids))
    
    for (u,i) in tqdm(df_ratings) :
        negatives = []
        negatives.append((u,i))
        for t in range(25) :
            j = np.random.randint(len(items))
            while (u,j) in zipped :
                j = np.random.randint(len(items))
            negatives.append(j)
        negativeList.append(negatives)
    df_neg = pd.DataFrame(negativeList)
    return df_neg

train_df_neg = get_negatives(train_uids,train_iids,novels,df_test)
real_df_neg = get_negatives(real_uids,real_iids,novels,df)

"""==========================================================================================="""

train_df_neg

100%|██████████| 1007/1007 [00:01<00:00, 993.64it/s]


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,"(2, 1)",0,1,2,2,2,1,1,1,1,...,25,25,0,2,1,25,2,2,25,0
1,"(3, 1)",0,11,0,11,1,11,1,11,1,...,1,1,1,0,11,11,0,11,11,1
2,"(4, 1)",21,0,0,0,1,1,0,0,1,...,0,0,0,1,0,0,21,21,21,1
3,"(5, 1)",0,8,8,18,18,8,0,1,18,...,8,18,1,18,0,18,1,1,0,18
4,"(6, 1)",0,1,1,0,1,0,1,0,1,...,1,1,1,0,0,1,0,0,1,0
5,"(7, 1)",0,17,28,17,1,17,28,0,28,...,28,28,1,0,1,1,1,28,17,0
6,"(8, 1)",0,0,1,1,1,10,10,0,1,...,1,0,0,0,1,10,10,0,1,10
7,"(9, 1)",7,7,7,0,0,0,1,0,1,...,1,0,0,0,0,7,0,1,7,7
8,"(10, 1)",0,0,0,0,1,1,1,0,0,...,0,0,1,0,1,0,0,1,0,1
9,"(11, 1)",1,0,0,1,0,0,0,0,0,...,0,0,1,1,0,0,0,1,0,1


In [6]:
len(users) , len(novels)

(30, 35)

In [7]:
users

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31]

In [8]:
max(users)

31

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

latent_features = 8

## input 데이터 정의

user = Input(shape = (1,), dtype = 'int32')
item = Input(shape = (1,), dtype = 'int32')

# 사용자 기반 GMF 임베딩 입력층 레이어 지정
gmf_user_embedding = Embedding(max(users)+1, latent_features, input_length=user.shape[1])(user)
gmf_user_embedding = Flatten()(gmf_user_embedding)

# Item embedding for GMF
gmf_item_embedding = Embedding(max(novels)+1, latent_features, input_length=item.shape[1])(item)
gmf_item_embedding = Flatten()(gmf_item_embedding)

# User embedding for MLP
mlp_user_embedding = Embedding(max(users)+1, 32, input_length=user.shape[1])(user)
mlp_user_embedding = Flatten()(mlp_user_embedding)

# Item embedding for MLP
mlp_item_embedding = Embedding(max(novels)+1, 32, input_length=item.shape[1])(item)
mlp_item_embedding = Flatten()(mlp_item_embedding)

# GMF layers
gmf_mul =  Multiply()([gmf_user_embedding, gmf_item_embedding])

# MLP layers
mlp_concat = Concatenate()([mlp_user_embedding, mlp_item_embedding])
mlp_dropout = Dropout(0.2)(mlp_concat)

# Layer1
mlp_layer_1 = Dense(units=64, activation='relu', name='mlp_layer1')(mlp_dropout)  # (64,1)
mlp_dropout1 = Dropout(rate=0.2, name='dropout1')(mlp_layer_1)                    # (64,1)
mlp_batch_norm1 = BatchNormalization(name='batch_norm1')(mlp_dropout1)            # (64,1)

# Layer2
mlp_layer_2 = Dense(units=32, activation='relu', name='mlp_layer2')(mlp_batch_norm1)  # (32,1)
mlp_dropout2 = Dropout(rate=0.2, name='dropout2')(mlp_layer_2)                        # (32,1)
mlp_batch_norm2 = BatchNormalization(name='batch_norm2')(mlp_dropout2)                # (32,1)

# Layer3
mlp_layer_3 = Dense(units=16, activation='relu', name='mlp_layer3')(mlp_batch_norm2)  # (16,1)

# Layer4
mlp_layer_4 = Dense(units=8, activation='relu', name='mlp_layer4')(mlp_layer_3)       # (8,1)

# merge GMF + MLP
merged_vector = tf.keras.layers.concatenate([gmf_mul, mlp_layer_4])

# Output layer
output_layer = Dense(1, kernel_initializer='lecun_uniform', name='output_layer')(merged_vector) # 1,1 / h(8,1)초기화

# Model
model = Model([user, item], output_layer)
model.compile(optimizer= 'adam', loss= 'binary_crossentropy')
model.build(user.shape)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 32)        1024        input_1[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 32)        1152        input_2[0][0]                    
______________________________________________________________________________________________

In [10]:
# 학습 데이터 생성 
def get_train_instances(uids, iids, num_neg, num_items) :
    """
    모델에 학습 시킬 수치화된 데이터 생성
    """
    user_input, item_input, labels = [], [], []
    zipped = set(zip(uids, iids))
    
    for (u,i) in zip(uids, iids) :
        
        # pos item 추가
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        
        # neg item 추가
        for t in range(num_neg) :
            j = np.random.randint(num_items)     # neg_item j num_neg 개 샘플링
            while (u,j) in zipped :             # u가 j를 이미 선택했다면 다시 샘플링
                j = np.random.randint(num_items)
            user_input.append(u) # [u1, u1,  u1,  ...]
            item_input.append(j) # [pos_i, neg_j1, neg_j2, ...]
            labels.append(0)     # [1, 0,  0,  ...]
    return user_input, item_input, labels

num_neg = 3

user_input, item_input, labels = get_train_instances(train_uids, train_iids, num_neg, len(novels))

print("샘플 유저 아이디 5개 : {}".format(user_input[:5]))
print("샘플 아이템 아이디 5개 : {}".format(item_input[:5]))
print("샘플 구매(평점리플) 여부 5개 : {}".format(labels[:5]))

샘플 유저 아이디 5개 : [3, 3, 3, 3, 4]
샘플 아이템 아이디 5개 : [2, 1, 11, 11, 2]
샘플 구매(평점리플) 여부 5개 : [1, 0, 0, 0, 1]


In [11]:
from sklearn.utils import shuffle

# numpy 형식으로 변환, 학습은 기본적으로 numpy 형식으로
# 데이터 섞어주기

user_data_shuff, item_data_shuff, label_data_shuff = shuffle(user_input,item_input,labels)

user_data_shuff = np.array(user_data_shuff).reshape(-1,1)
item_data_shuff = np.array(item_data_shuff).reshape(-1,1)
label_data_shuff = np.array(label_data_shuff).reshape(-1,1)

print("유저 아이디 5개 : {}".format(user_data_shuff[:5]))
print("아이템 아이디 5개 : {}".format(item_data_shuff[:5]))
print("구매(평점리플) 여부 5개 : {}".format(label_data_shuff[:5]))

유저 아이디 5개 : [[23]
 [11]
 [ 8]
 [23]
 [30]]
아이템 아이디 5개 : [[ 0]
 [31]
 [27]
 [19]
 [ 1]]
구매(평점리플) 여부 5개 : [[0]
 [1]
 [1]
 [1]
 [0]]


In [12]:
## 모델 학습

model.fit([user_data_shuff, item_data_shuff], label_data_shuff, epochs = 10, batch_size = 64, verbose = 1)

Epoch 1/10
62/62 [==============================] - 2s 7ms/step - loss: 1.4265
Epoch 2/10
62/62 [==============================] - 0s 7ms/step - loss: 0.6387
Epoch 3/10
62/62 [==============================] - 0s 7ms/step - loss: 0.5059
Epoch 4/10
62/62 [==============================] - 0s 7ms/step - loss: 0.4206
Epoch 5/10
62/62 [==============================] - 0s 7ms/step - loss: 0.4022
Epoch 6/10
62/62 [==============================] - 0s 8ms/step - loss: 0.3693
Epoch 7/10
62/62 [==============================] - 0s 7ms/step - loss: 0.3089
Epoch 8/10
62/62 [==============================] - 0s 7ms/step - loss: 0.2749
Epoch 9/10
62/62 [==============================] - 1s 9ms/step - loss: 0.2422
Epoch 10/10
62/62 [==============================] - 0s 8ms/step - loss: 0.2089


In [13]:
import heapq

def get_hits(k_ranked, holdout) :
    """
    hit 생성 함수
    hit := holdout(어떤 유저의 positive item)이 예측된 목록의 K 순위 내에 있는지의 여부
    """
    for item in k_ranked :
        if item == holdout :
            return 1
    return 0

def eval_rating(idx, ratings, negetives, K, model) :
    """
    모든 유저의 예측 목록에서 각 유저마다 positive item이 K 순위 내에 얼마나 많이 있는지 평가하는 함수
    """
    items = negetives[idx]          # negative items [neg_item_id, ... ] (1,100)
    user_idx = ratings[idx][0]      # [user_id, item_id][0]
    holdout = ratings[idx][1]       # [user_id, item_id][1]
    items.append(holdout)           # holdout 추가 [neg_item_id, ..., holdout] (1,101)
    
    # prediction
    
    predict_user = np.full(len(items), user_idx, dtype='int32').reshape(-1,1) # [[user_id], ...], (101, 1)
    np_items = np.array(items).reshape(-1,1)                                  # [[item_id], ... ], (101, 1)
    
    predictions = model.predict([predict_user, np_items])
    predictions = predictions.flatten().tolist()
    item_to_pre_score = {item:pre for item,pre in zip(items, predictions)}
    
    # 점수가 높은 상위 K개 아이템 리스트 생성
    k_ranked = heapq.nlargest(K, item_to_pre_score, key = item_to_pre_score.get)
    
    # holdout이 상위 K 순위에 포함 되는지 체크
    # { 1 : 포함 , 0 : 안포함}
    hits = get_hits(k_ranked, holdout)
    
    return hits

def evaluate_top_k(df_neg, df_test, model, K=5) :
    """
    TOP-K metric을 사용해 모델을 평가하는 함수
    """
    hits = []
    test_u = df_test['user_id'].values.tolist()
    test_i = df_test['novel_id'].values.tolist()
    
    test_ratings = list(zip(test_u, test_i))
    df_neg = df_neg.drop(df_neg.columns[0], axis = 1)
    test_negetives = df_neg.values.tolist() #[[(user_id, item_id =holdout)], neg_item,neg_item,....]
    
    # user 샘플링
#     sample_idx_lst = np.random.choice(len(test_ratings), int(len(test_ratings) * 0.3))
    for user_idx in tqdm(range(len(test_ratings))) :
        hitrate = eval_rating(user_idx, test_ratings, test_negetives, K, model)
        hits.append(hitrate) # ex. [1,0,1,1,0,....] (1, df_test.shape[0])
    
    return hits

def calculate_top_k_metric(df_neg,df_test,model) :
    hit_lst = evaluate_top_k(df_neg,df_test,model,K=5)
    top_k_metric = np.mean(hit_lst)
    print("metric : {}".format(top_k_metric))
    return top_k_metric

In [14]:
# 평가 
# 평가 방식 모델의 추천 리스트 속에 유저가 실제로 본 소설이 있는지 (a =  있으면 1 없으면 0 ) 
#                                                              (b = 전체 수 )
#                                                              (a/b)

top_k_metric = calculate_top_k_metric(train_df_neg,df_test,model)

100%|██████████| 30/30 [00:01<00:00, 25.15it/s]

metric : 1.0


In [1]:
# import client
# # users, novels, df_test, train_uids, train_nids, real_uids, real_nids,\
# # train_df_neg, real_df_neg = client.Load_Dataset().ncf_export_df()
# # model = client.Run().train_model()

In [2]:
result = client.Content_based_model().create_recommend_data_similarity()

  0%|          | 0/35 [00:00<?, ?it/s]

In [2]:
import tensorflow as tf
model = tf.keras.models.load_model('./data/NCF_model.h5')

In [3]:
result2 = client.Create_Base_NeuMF().export_recommend_users_prediction(model)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/1007 [00:00<?, ?it/s]

In [4]:
result2

,user_id,recommend_novel,predictions_score
0,2,25,-0.223346
1,2,0,-0.349688
2,2,2,-0.422046
0,3,11,0.005148
1,3,0,-0.676554
...,...,...,...
0,29,0,-0.648760
0,30,26,-0.222849
1,30,27,-0.325593
2,30,0,-0.417974


In [10]:
result2

result2.reset_index(inplace = True, drop = True)

In [13]:
result2

dict_ = result2.T.to_dict()
dict_

{0: {'user_id': 2.0,
  'recommend_novel': 25.0,
  'predictions_score': -0.22334593534469604},
 1: {'user_id': 2.0,
  'recommend_novel': 0.0,
  'predictions_score': -0.34968817234039307},
 2: {'user_id': 2.0,
  'recommend_novel': 2.0,
  'predictions_score': -0.422046035528183},
 3: {'user_id': 3.0,
  'recommend_novel': 11.0,
  'predictions_score': 0.0051475465297698975},
 4: {'user_id': 3.0,
  'recommend_novel': 0.0,
  'predictions_score': -0.6765538454055786},
 5: {'user_id': 4.0,
  'recommend_novel': 21.0,
  'predictions_score': -0.1428021788597107},
 6: {'user_id': 4.0,
  'recommend_novel': 0.0,
  'predictions_score': -0.5026809573173523},
 7: {'user_id': 5.0,
  'recommend_novel': 8.0,
  'predictions_score': 0.11458224803209305},
 8: {'user_id': 5.0,
  'recommend_novel': 18.0,
  'predictions_score': -0.13798141479492188},
 9: {'user_id': 5.0,
  'recommend_novel': 0.0,
  'predictions_score': -0.28037595748901367},
 10: {'user_id': 6.0,
  'recommend_novel': 0.0,
  'predictions_score': 

In [14]:
list(dict_.values())

[{'user_id': 2.0,
  'recommend_novel': 25.0,
  'predictions_score': -0.22334593534469604},
 {'user_id': 2.0,
  'recommend_novel': 0.0,
  'predictions_score': -0.34968817234039307},
 {'user_id': 2.0,
  'recommend_novel': 2.0,
  'predictions_score': -0.422046035528183},
 {'user_id': 3.0,
  'recommend_novel': 11.0,
  'predictions_score': 0.0051475465297698975},
 {'user_id': 3.0,
  'recommend_novel': 0.0,
  'predictions_score': -0.6765538454055786},
 {'user_id': 4.0,
  'recommend_novel': 21.0,
  'predictions_score': -0.1428021788597107},
 {'user_id': 4.0,
  'recommend_novel': 0.0,
  'predictions_score': -0.5026809573173523},
 {'user_id': 5.0,
  'recommend_novel': 8.0,
  'predictions_score': 0.11458224803209305},
 {'user_id': 5.0,
  'recommend_novel': 18.0,
  'predictions_score': -0.13798141479492188},
 {'user_id': 5.0,
  'recommend_novel': 0.0,
  'predictions_score': -0.28037595748901367},
 {'user_id': 6.0,
  'recommend_novel': 0.0,
  'predictions_score': -0.6962177157402039},
 {'user_id':

In [6]:
result2.to_dict(orient='list').keys()

dict_keys(['user_id', 'recommend_novel', 'predictions_score'])

In [7]:
result

{'trains[]': '{"origin_novel_id": 1, "recommend_novel_id": 3, "predictions": 0.8857052812270015}',
 'trains[]': '{"origin_novel_id": 1, "recommend_novel_id": 19, "predictions": 0.8809142559762349}',
 'trains[]': '{"origin_novel_id": 1, "recommend_novel_id": 9, "predictions": 0.8787823917507073}',
 'trains[]': '{"origin_novel_id": 1, "recommend_novel_id": 10, "predictions": 0.8688765787109326}',
 'trains[]': '{"origin_novel_id": 1, "recommend_novel_id": 33, "predictions": 0.8675687445683998}',
 'trains[]': '{"origin_novel_id": 2, "recommend_novel_id": 6, "predictions": 0.8590286269584168}',
 'trains[]': '{"origin_novel_id": 2, "recommend_novel_id": 24, "predictions": 0.8387787981443832}',
 'trains[]': '{"origin_novel_id": 2, "recommend_novel_id": 25, "predictions": 0.8183345521672619}',
 'trains[]': '{"origin_novel_id": 2, "recommend_novel_id": 27, "predictions": 0.8135268255419498}',
 'trains[]': '{"origin_novel_id": 2, "recommend_novel_id": 3, "predictions": 0.8095108977119105}',
 'tr

In [8]:
result2

,user_id,recommend_novel,predictions_score
0,2,25,-0.223346
1,2,0,-0.349688
2,2,2,-0.422046
0,3,11,0.005148
1,3,0,-0.676554
...,...,...,...
0,29,0,-0.648760
0,30,26,-0.222849
1,30,27,-0.325593
2,30,0,-0.417974


In [12]:
rjson

{'result': 'success',
 'data': [{'novel_id': 1, 'user_id': 2, 'grade': 5},
  {'novel_id': 1, 'user_id': 3, 'grade': 4.5},
  {'novel_id': 1, 'user_id': 4, 'grade': 4},
  {'novel_id': 1, 'user_id': 5, 'grade': 4},
  {'novel_id': 1, 'user_id': 6, 'grade': 5},
  {'novel_id': 1, 'user_id': 7, 'grade': 9},
  {'novel_id': 1, 'user_id': 8, 'grade': 2},
  {'novel_id': 1, 'user_id': 9, 'grade': 7.75},
  {'novel_id': 1, 'user_id': 10, 'grade': 3},
  {'novel_id': 1, 'user_id': 11, 'grade': 3.3333333333333335},
  {'novel_id': 1, 'user_id': 12, 'grade': 2},
  {'novel_id': 1, 'user_id': 13, 'grade': 5.75},
  {'novel_id': 1, 'user_id': 14, 'grade': 3.5},
  {'novel_id': 1, 'user_id': 16, 'grade': 7.5},
  {'novel_id': 1, 'user_id': 17, 'grade': 4.333333333333333},
  {'novel_id': 1, 'user_id': 18, 'grade': 6.5},
  {'novel_id': 1, 'user_id': 19, 'grade': 2},
  {'novel_id': 1, 'user_id': 20, 'grade': 9},
  {'novel_id': 1, 'user_id': 21, 'grade': 3},
  {'novel_id': 1, 'user_id': 22, 'grade': 2.5},
  {'novel

{'result': 'suceess',
 'data': ['{"origin_novel_id": 1, "recommend_novel_id": 3, "predictions": 0.8857052812270015}',
  '{"origin_novel_id": 1, "recommend_novel_id": 19, "predictions": 0.8809142559762349}',
  '{"origin_novel_id": 1, "recommend_novel_id": 9, "predictions": 0.8787823917507073}',
  '{"origin_novel_id": 1, "recommend_novel_id": 10, "predictions": 0.8688765787109326}',
  '{"origin_novel_id": 1, "recommend_novel_id": 33, "predictions": 0.8675687445683998}',
  '{"origin_novel_id": 2, "recommend_novel_id": 6, "predictions": 0.8590286269584168}',
  '{"origin_novel_id": 2, "recommend_novel_id": 24, "predictions": 0.8387787981443832}',
  '{"origin_novel_id": 2, "recommend_novel_id": 25, "predictions": 0.8183345521672619}',
  '{"origin_novel_id": 2, "recommend_novel_id": 27, "predictions": 0.8135268255419498}',
  '{"origin_novel_id": 2, "recommend_novel_id": 3, "predictions": 0.8095108977119105}',
  '{"origin_novel_id": 3, "recommend_novel_id": 13, "predictions": 0.901283526795926

In [40]:
result

{'trains[]': '{"origin_novel_id": 1, "recommend_novel_id": 3, "predictions": 0.8857052812270015}',
 'trains[]': '{"origin_novel_id": 1, "recommend_novel_id": 19, "predictions": 0.8809142559762349}',
 'trains[]': '{"origin_novel_id": 1, "recommend_novel_id": 9, "predictions": 0.8787823917507073}',
 'trains[]': '{"origin_novel_id": 1, "recommend_novel_id": 10, "predictions": 0.8688765787109326}',
 'trains[]': '{"origin_novel_id": 1, "recommend_novel_id": 33, "predictions": 0.8675687445683998}',
 'trains[]': '{"origin_novel_id": 2, "recommend_novel_id": 6, "predictions": 0.8590286269584168}',
 'trains[]': '{"origin_novel_id": 2, "recommend_novel_id": 24, "predictions": 0.8387787981443832}',
 'trains[]': '{"origin_novel_id": 2, "recommend_novel_id": 25, "predictions": 0.8183345521672619}',
 'trains[]': '{"origin_novel_id": 2, "recommend_novel_id": 27, "predictions": 0.8135268255419498}',
 'trains[]': '{"origin_novel_id": 2, "recommend_novel_id": 3, "predictions": 0.8095108977119105}',
 'tr